In [70]:
import pandas as pd
df = pd.read_excel("AG.xlsx", sheet_name=None,header=0)
print(df)

{'2014':    dze la  vks-ch- vkbZVe la-   eq[; 'kh"kZ     o"kZ   jkf'k :Ik;s esa   \
0     1.0                 20.0        2014.0  2022-23            504568   
1     2.0                 21.0        2014.0  2022-23            459807   
2     3.0                 22.0        2014.0  2022-23            359668   
3     4.0                 24.0        2014.0  2022-23            390827   
4     5.0                 25.0        2014.0  2023-24            243156   
5     6.0                 26.0        2014.0  2023-24            371128   
6     7.0                 27.0        2014.0  2023-24            374829   
7     8.0                 28.0        2014.0  2023-24            249400   
8     NaN                  NaN           NaN      NaN           2953383   

              vk{ksi dk fooj.k   \
0  ikoj QsDVj ljpktZ dk Hkqxrku   
1  ikoj QsDVj ljpktZ dk Hkqxrku   
2  ikoj QsDVj ljpktZ dk Hkqxrku   
3  ikoj QsDVj ljpktZ dk Hkqxrku   
4  ikoj QsDVj ljpktZ dk Hkqxrku   
5  ikoj QsDVj ljpktZ dk Hkqxrk

In [71]:
type(df)

dict

In [72]:
c_df = pd.DataFrame()
for key,value in df.items():
    if key not in ['Sheet1']:
        c_df =c_df.append(value.iloc[:,2:5], ignore_index=True)
    
    


In [73]:
c_df.iloc[:,1] = c_df.iloc[:,1].str.replace('&','-')


In [74]:
c_df.iloc[:,0:3]

,"eq[; 'kh""kZ","o""kZ",jkf'k :Ik;s esa
0,2014.0,2022-23,504568.0
1,2014.0,2022-23,459807.0
2,2014.0,2022-23,359668.0
3,2014.0,2022-23,390827.0
4,2014.0,2023-24,243156.0
...,...,...,...
274,2070.0,2019-20,201094.0
275,2070.0,2019-20,257145.0
276,2070.0,2020-21,660000.0
277,2070.0,2020-21,515007.0


In [75]:
c_df.set_axis(["A", "B", "C", "D", "E", "F"], axis="columns", inplace=True)
c_df.iloc[:,0:3]

,A,B,C
0,2014.0,2022-23,504568.0
1,2014.0,2022-23,459807.0
2,2014.0,2022-23,359668.0
3,2014.0,2022-23,390827.0
4,2014.0,2023-24,243156.0
...,...,...,...
274,2070.0,2019-20,201094.0
275,2070.0,2019-20,257145.0
276,2070.0,2020-21,660000.0
277,2070.0,2020-21,515007.0


In [76]:
import numpy as np
years = c_df.B.unique()
years_list =[]
for year in years :
    if str(year) != 'nan':
        years_list.append(year)
        
years_list.sort()
sheets_df_dic={}
for year in years_list:
    sheet_df = c_df.loc[c_df['B'] == year]
    sheet_df.reset_index(inplace = True) 
    sheet_df = sheet_df.iloc[:,[1,3]]
    sheets_df_dic[year] = sheet_df


In [77]:

writer = pd.ExcelWriter("Final_new.xlsx", engine="xlsxwriter")
workbook = writer.book
merge_format = workbook.add_format({'align': 'center', 'valign': 'vcenter', 'border': 1,'bold' : True})
total_format = workbook.add_format({
    'font_size': 12,
    'bold' : True
})
    
for idx,key in enumerate(sheets_df_dic):
    sheets_df_dic[key].to_excel(writer, sheet_name='Sheet2', startcol=idx*4,startrow=2 ,index=False, header=False)
    worksheet = writer.sheets['Sheet2']
    worksheet.merge_range(0, idx*4, 0, idx*4+2, key, merge_format)
    column_sum = sheets_df_dic[key].iloc[:,[1]].sum()
    column_list = list(sheets_df_dic[key].iloc[:,0])
    values_list =[]
    for column in column_list:
        if str(column)!='nan':
            values_list.append(column)
    row_no = len(values_list)+2    
        
    worksheet.write(row_no,idx*4, 'Total', total_format)
    worksheet.write(row_no,idx*4+1, column_sum, total_format)
    
writer.save()
writer.close()    


In [217]:
df3 = pd.read_excel("Final_new.xlsx")
times = 10
dict_new = {}
for i in range(times):
    n = i*4
    current_df = df3.iloc[:, [ n,n+1 ]]
    dict_new[f'old_df{i+1}'] = current_df
    
df4 = pd.read_excel("Final_new2.xlsx")
times = 10
dict_new2 = {}
for i in range(times):
    n = i*4
    current_df = df4.iloc[:, [ n,n+1 ]]
    dict_new2[f'old_df{i+1}'] = current_df

def base(dict1):
    column_list = list(dict1.iloc[:,0])
    values_list =[]
    for column in column_list:
        if str(column)!='nan':
            values_list.append(column)
    row_nu = len(values_list)
    return row_nu
new_df = pd.DataFrame()
writer = pd.ExcelWriter("Main_file.xlsx", engine="xlsxwriter")
workbook = writer.book
merge_format = workbook.add_format({'align': 'center', 'valign': 'vcenter', 'border': 1,'bold' : True})
idx = 0
for key in dict_new:
    row_no = base(dict_new[key])
    row_no2 = base(dict_new2[key])
    v1 = dict_new[key]
    v2 = dict_new2[key]
    v1 = v1.drop(row_no)
    v2 = v2.drop(row_no2)
    final_df = pd.concat([v1, v2])
    final2 = final_df.drop_duplicates(keep=False)   
    final2.to_excel(writer, sheet_name='Sheet1', startcol=idx*2,startrow=0 ,index=False)
    worksheet.merge_range(0, idx*2, 0, idx*2+2, key, merge_format)
    idx = idx+1
    
    
writer.save()
writer.close()    

In [165]:
a_df = c_df.loc[c_df['B'] == '2020-21']
a_df.reset_index(inplace = True) 
a_df = a_df.iloc[:,1:4]
a_df.to_excel('File4.xlsx',index=False)

In [80]:
b_df = c_df.loc[c_df['B'] == '2023-24']
b_df.reset_index(inplace = True) 
b_df = b_df.iloc[:,1:4]

In [81]:
b_df.to_excel('File1.xlsx',index=False)

In [82]:
d_df = c_df.loc[c_df['B'] == '2022-23']
d_df.reset_index(inplace = True) 
d_df = d_df.iloc[:,1:4]
d_df.to_excel('File2.xlsx',index=False)

In [83]:
e_df = c_df.loc[c_df['B'] == '2021-22']
e_df.reset_index(inplace = True) 
e_df = e_df.iloc[:,1:4]
e_df.to_excel('File3.xlsx',index=False)


In [84]:
f_df = c_df.loc[c_df['B'] == '2019-20']
f_df.reset_index(inplace = True) 
f_df = f_df.iloc[:,1:4]
f_df.to_excel('File5.xlsx',index=False)

In [85]:
g_df = c_df.loc[c_df['B'] == '2018-19']
g_df.reset_index(inplace = True) 
g_df = g_df.iloc[:,1:4]
g_df.to_excel('File6.xlsx',index=False)

In [86]:
h_df = c_df.loc[c_df['B'] == '2017-18']
h_df.reset_index(inplace = True) 
h_df = h_df.iloc[:,1:4]
h_df.to_excel('File7.xlsx',index=False)

In [87]:
i_df = c_df.loc[c_df['B'] == '2016-17']
i_df.reset_index(inplace = True) 
i_df = i_df.iloc[:,1:4]
i_df.to_excel('File8.xlsx',index=False)

In [88]:
j_df = c_df.loc[c_df['B'] == '2015-16']
j_df.reset_index(inplace = True) 
j_df = j_df.iloc[:,1:4]
j_df.to_excel('File9.xlsx',index=False)

In [89]:
k_df = c_df.loc[c_df['B'] == '2014-15']
k_df.reset_index(inplace = True) 
k_df = k_df.iloc[:,1:4]
k_df.to_excel('File_10.xlsx',index=False)


In [90]:
import os
path ='C:/Users/aviri/'
files = os.listdir(path)
files_xlsx = [f for f in files if f[-4:] == 'xlsx']


In [91]:
mydf_list = [pd.read_excel(os.path.join(path, f)) for f in files_xlsx]
mydf = pd.concat(mydf_list, axis = 1)


In [92]:
writer = pd.ExcelWriter("Final.xlsx", engine="xlsxwriter")

In [93]:
e_df.to_excel(writer, sheet_name='Sheet2',startrow=2, index=False, header=False)  
b_df.to_excel(writer, sheet_name='Sheet2', startcol=4,startrow=2 ,index=False, header=False)
d_df.to_excel(writer, sheet_name='Sheet2', startcol=8,startrow=2,index=False, header=False)
a_df.to_excel(writer, sheet_name='Sheet2', startcol=12,startrow=2,index=False, header=False)
f_df.to_excel(writer, sheet_name='Sheet2', startcol=16,startrow=2,index=False, header=False)
g_df.to_excel(writer, sheet_name='Sheet2', startcol=20,startrow=2,index=False, header=False)
h_df.to_excel(writer, sheet_name='Sheet2', startcol=24,startrow=2,index=False, header=False)
i_df.to_excel(writer, sheet_name='Sheet2', startcol=28,startrow=2,index=False, header=False)
j_df.to_excel(writer, sheet_name='Sheet2', startcol=32,startrow=2,index=False, header=False)
k_df.to_excel(writer, sheet_name='Sheet2', startcol=36,startrow=2,index=False, header=False)

In [94]:
workbook = writer.book
worksheet = writer.sheets['Sheet2']
merge_format = workbook.add_format({'align': 'center', 'valign': 'vcenter', 'border': 1})

In [95]:
worksheet.merge_range(0, 0, 0, 2, "2021-22", merge_format)
worksheet.merge_range(0, 4, 0, 6, "2023-24", merge_format)
worksheet.merge_range(0, 8, 0, 10, "2022-23", merge_format)
worksheet.merge_range(0, 12, 0, 14, "2020-21", merge_format)
worksheet.merge_range(0, 16, 0, 18, "2019-20", merge_format)
worksheet.merge_range(0, 20, 0, 22, "2018-19", merge_format)
worksheet.merge_range(0, 24, 0, 26, "2017-18", merge_format)
worksheet.merge_range(0, 28, 0, 30, "2016-17", merge_format)
worksheet.merge_range(0, 32,0, 34, "2015-16", merge_format)
worksheet.merge_range(0, 36, 0, 38, "2014-15", merge_format)

In [96]:
datetime_format = workbook.add_format({
    'font_size': 12
})
worksheet.write('A13', 'Total', datetime_format)
worksheet.write('B13', 'Total', datetime_format)



0

In [97]:
writer.save()
writer.close()

In [105]:
df3 = pd.read_excel("Final_new.xlsx")
times = 10
dict_new = {}
for i in range(times):
    n = i*4
    current_df = df3.iloc[:, [ n,n+1 ]]
    dict_new[f'old_df{i+1}'] = current_df

df4 = pd.read_excel("Final_new2.xlsx")
times = 10
dict_new2 = {}
for i in range(times):
    n = i*4
    current_df = df4.iloc[:, [ n,n+1 ]]
    dict_new2[f'old_df{i+1}'] = current_df
for key in dict_new:
    v1 = dict_new[key]
    v2 = dict_new2[key]
    final_df = pd.concat([v1, v2])
    final2 = final_df.drop_duplicates(keep=False)
    

52


In [99]:
dict([('sape', 4139), ('guido', 4127), ('jack', 4098)])
a  = dict([('sape', 4139)])
type(a)

dict

In [100]:
a = lambda x,y: x + y+ 1
a(2,3)

6

In [188]:
import pandas as pd

# Creating the first Dataframe using dictionary
df1 = df = pd.DataFrame({"a": [1, 2, 3, 4],
						"b": [5, 6, 7, 8]})

# Append Dict as row to DataFrame
new_row = {"c": 10, "d": 10}
df2 = df.append(new_row, ignore_index=True)

print(df2)


     a    b     c     d
0  1.0  5.0   NaN   NaN
1  2.0  6.0   NaN   NaN
2  3.0  7.0   NaN   NaN
3  4.0  8.0   NaN   NaN
4  NaN  NaN  10.0  10.0


In [ ]:
import .project1 from base
